In [1]:
import pandas as pd

In [2]:
LABEL = "./data/covid_labels.csv"

In [8]:
label = pd.read_csv(LABEL)
label

,Id,Predicted
0,0,0
1,1,0
2,2,1
3,3,0
4,4,1
...,...,...
17453,17453,1
17454,17454,0
17455,17455,1
17456,17456,0


In [9]:
covid_df = pd.read_csv("./data/covid_bert_data.csv")
covid_df

,Unnamed: 0,source,replies
0,0,"According to the New York Times, Warner Bros. ...",NaN
1,1,Hurricane Hanna has made landfall in Texas.\n\...,@TexasTribune Guess what the cause of death is...
2,2,Monkeys on the loose in India with stolen coro...,NaN
3,3,NaN,@BelAkinyii When was the last time you washed ...
4,4,"“If Trump felt comfortable having it here, the...",@HeidiNBC These Trump fans have a right to att...
...,...,...,...
17453,17453,I wonder how many lives could’ve been saved if...,@funder Wonder how many lives could have been ...
17454,17454,The @thetimes front page on 17th March. The fi...,@NadineDorries @thetimes Inadequate supplies o...
17455,17455,Trump just completed the racism trifecta in a ...,@DNCWarRoom Fact check: Chinese is not a race....
17456,17456,Here are a few of my photographs from today’s ...,@Jess__Taylor__ @davidallengreen Eck! What are...


In [14]:
covid_df["label"] = label["Predicted"]
covid_df

,Unnamed: 0,source,replies,label
0,0,"According to the New York Times, Warner Bros. ...",NaN,0
1,1,Hurricane Hanna has made landfall in Texas.\n\...,@TexasTribune Guess what the cause of death is...,0
2,2,Monkeys on the loose in India with stolen coro...,NaN,1
3,3,NaN,@BelAkinyii When was the last time you washed ...,0
4,4,"“If Trump felt comfortable having it here, the...",@HeidiNBC These Trump fans have a right to att...,1
...,...,...,...,...
17453,17453,I wonder how many lives could’ve been saved if...,@funder Wonder how many lives could have been ...,1
17454,17454,The @thetimes front page on 17th March. The fi...,@NadineDorries @thetimes Inadequate supplies o...,0
17455,17455,Trump just completed the racism trifecta in a ...,@DNCWarRoom Fact check: Chinese is not a race....,1
17456,17456,Here are a few of my photographs from today’s ...,@Jess__Taylor__ @davidallengreen Eck! What are...,0
